## 초기 system prompt 설정 및 벡터 store 생성

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from random import randint, choice
import json
from openai import OpenAI
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# 벡터 DB 세팅
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

## 초기 질문 리스트 가져오기

In [14]:
import json

def load_questions_from_jsonl_minimal(path: str, key: str = "question"):
    """JSONL에서 각 라인의 `question` 값만 순서/중복/공백 그대로 추출"""
    questions = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            obj = json.loads(line)
            if isinstance(obj, dict) and key in obj:
                questions.append(obj[key])
    return questions

# 사용 예시
initial_questions = load_questions_from_jsonl_minimal("./question.jsonl")
initial_questions


['프로젝트의 현재 완료율이 65%인 이유는 무엇인가요?',
 '기술적 문제 해결을 위해 어떤 추가 리소스가 투입될 예정인가요?',
 '홍길동의 개인 사정은 어떤 영향을 미쳤으며, 그에 대한 지원 방안은 무엇인가요?',
 '베타 버전 출시 일정이 2주 지연된 이유는 무엇인가요?',
 '테스트 단계에서 어떤 특정 모듈의 성능 저하가 발생했나요?',
 'QA팀의 테스트 지원 요청에 대한 구체적인 내용은 무엇인가요?',
 '각 팀원이 매주 작업 진행 상황을 업데이트하는 방법은 어떤 방식으로 이루어지나요?',
 '외부 협력업체와의 일정 조율은 어떻게 진행되고 있나요?',
 '프로젝트 진행 상황을 주간 회의에서 어떻게 공유할 계획인가요?',
 '리스크 관리 섹션을 강화하기 위한 구체적인 방안은 무엇인가요?',
 '기술적 문제 해결을 위한 개발팀의 협의는 어떤 형태로 이루어지고 있나요?',
 '추가 리소스 투입 후 성과 점검은 어떤 기준으로 진행되나요?',
 '테스트 계획서의 내용은 어떻게 구성되어 있으며, 어떤 항목들이 포함되나요?',
 '사용자 피드백을 어떻게 수집하고 분석할 계획인가요?',
 '프로젝트 범위에 포함된 교육 콘텐츠는 어떤 형태로 제공될 예정인가요?',
 '향후 개정 이력에 대한 관리 방안은 어떻게 될 예정인가요?',
 '프로젝트의 성공적인 완료를 위해 필요한 추가적인 지원은 무엇인가요?',
 '특정 모듈의 성능 저하가 전체 플랫폼에 어떤 식으로 영향을 미칠 수 있나요?',
 '각 단계별 진척 상황을 어떻게 기록하고 관리할 예정인가요?',
 '홍길동 외에 다른 팀원들도 작업 지연이 발생하고 있나요?']

In [13]:
SYSTEM_PROMPT = {
    "role": "system",
    "content": "당신은 회사 내의 사내 문서 검색에 대한 정확한 답변을 하는 챗봇 어시스턴트 입니다."
               "대답은 항상 공손한 말투로 해야 합니다. "
               "그리고 검색 결과로만 답변하세요. 문서 밖 지식 금지, 없으면 '해당 질문에 대한 내용이 없습니다.'라고 답하세요. 각 문장/문단 끝에 [[refN]] 인용을 남기세요."
}

def generate_one_chat_sample(initial_q: str, max_turns: int = 5):
    messages = [SYSTEM_PROMPT]
    current_q = initial_q
    for turn in range(randint(2, max_turns + 1)):
        # (1) User 질문 추가
        messages.append({"role": "user", "content": current_q})

        # (2) 문서 검색 (vectorstore에서)
        docs = vectorstore.similarity_search(current_q, k=3)
        ref_text = "\n".join([f"{doc.page_content} [[ref{idx+1}]]" for idx, doc in enumerate(docs)])
        messages.append({"role": "system", "content": f"검색 결과:\n-----\n{ref_text}"})

        # (3) assistant 답변 (GPT로 RAG)
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.
        )
        assistant_reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})

        # 다음 질문 생성 여부 랜덤 (5턴이 max면 4턴 미만일때만 질문 생성)
        if turn < max_turns - 1 and choice([True, False]):
            # (4) GPT에게 다음 질문 생성 요청
            followup = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "앞선 대화를 기반으로 자연스러운 후속 질문 하나만 만들어줘."},
                    *messages
                ],
                temperature=0.5 # 후속 질문 생성을 위해서 temperature 올림
            )
            current_q = followup.choices[0].message.content.strip()
        else:
            break

    return {"messages": messages}


In [12]:
train_dataset = []
for q in initial_questions:
    sample = generate_one_chat_sample(q, max_turns=5)
    train_dataset.append(sample)

# 저장
with open("train_dataset2.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)
